In [1]:
### System Requirement ###
# At least 32 GB RAM (Cpu)

# Set-up the necessary data for the experiments in playground.ipynb

In [4]:
dataset = "binary_waterbirds"
device = 'cuda'
model_name = 'ViT-B-32' # 'ViT-H-14'
seed = 0
num_last_layers_ = 4
subset_dim = None
tot_samples_per_class = None
dataset_text_name = "top_1500_nouns_5_sentences_imagenet_clean"
datataset_image_name = "binary_waterbirds"
algorithm = "svd_data_approx"
path = './datasets'

# Additional params for LLAVA
full_output = False # If we want intermediate hiddn tokens, def False
vision_proj = True # If we want to project in shared space, def True

if model_name == "ViT-H-14":
    pretrained = "laion2B-s32B-b79K"
    precision = "fp32"
elif model_name == "ViT-L-14":
    pretrained = "laion2B-s32B-b82K"
    precision = "fp32"
elif model_name == "ViT-B-16":
    pretrained = "laion2B-s34B-b88K"
    precision = "fp32"
elif model_name == "ViT-B-32":
    pretrained = "laion2B-s34B-b79K"
    precision = "fp32"
elif model_name == "ViT-L-14-336":
    pretrained = "openai"
    precision = "fp16"

In [6]:
# Retrieve the activations (hooks) for each head in each layer, of a given ViT-model.
# Opt.: Setup a seed and use only a subset of the dataset (Imagenet)
!python -m utils.scripts.compute_activation_values --dataset {dataset} --device {device} --model {model_name} --pretrained {pretrained} --seed {seed} --batch_size 1  --samples_per_class {subset_dim} --tot_samples_per_class {tot_samples_per_class} --quantization {precision} --cache_dir "../cache/" --full_output {full_output} --vision_proj {vision_proj} 

Using local files
Model parameters: 151,277,313
Context length: 77
Vocab size: 49408
Len of res: 12
Full dataset
/cluster/apps/vogtlab/users/vstrozzi/software/anaconda/envs/MT/lib/python3.10/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
We are using a dataset containing 5794 images.
100%|███████████████████████████████████████| 5794/5794 [02:20<00:00, 41.28it/s]

Concatenating chunk files into final .npy arrays...
['./output_dir/binary_waterbirds_attn_ViT-B-32_seed_0_chunk0.npy', './output_dir/binary_waterbirds_attn_ViT-B-32_seed_0_chunk1.npy', './output_dir/binary_waterbirds_attn_ViT-B-32_

In [3]:
# Use the previous outputs (i.e. all the activations) to derive the final clip embeddings for the image dataset.
!python -m utils.scripts.compute_images_embedding --dataset {dataset} --model {model_name} --seed {seed}

(10000, 12, 12, 512) (10000, 13, 512)


In [4]:
# Derive the CLIP-embeddings of the classes' labels of a given dataset (i.e. allow zero-shot classifications)
!python -m utils.scripts.compute_classes_embeddings --dataset {dataset} --device {device} --model {model_name} --pretrained {pretrained} --cache_dir "../cache/"

Using local files
Model parameters: 151,277,313
Context length: 77
Vocab size: 49408
100%|███████████████████████████████████████| 1000/1000 [00:18<00:00, 55.45it/s]


In [5]:
# Given a dataset of text compute their CLIP's embeddings.
!python -m utils.scripts.compute_text_embeddings --device {device} --model {model_name} --pretrained {pretrained} --cache_dir "../cache/" --data_path "utils/text_descriptions/{dataset_text_name}.txt"

Using local files
Model parameters: 151,277,313
Context length: 77
Vocab size: 49408
100%|█████████████████████████████████████████████| 8/8 [00:05<00:00,  1.59it/s]


# Test text explanations

In [6]:
# Test an algorithm to explain the CLIP-embeddings with text (ours: svd_data_approx, their: text_span)
!python -m utils.scripts.compute_text_explanations --device {device} --model {model_name} --algorithm svd_data_approx --dataset {dataset} --seed {seed} --num_of_last_layers 4 --text_descriptions {dataset_text_name}

Number of layers: 12
  0%|                                                     | 0/4 [00:00<?, ?it/s]
Layer [8], Head: 0

Layer [8], Head: 1

Layer [8], Head: 2

Layer [8], Head: 3

Layer [8], Head: 4

Layer [8], Head: 5

Layer [8], Head: 6

Layer [8], Head: 7

Layer [8], Head: 8

Layer [8], Head: 9

Layer [8], Head: 10

Layer [8], Head: 11
 25%|███████████▎                                 | 1/4 [00:09<00:27,  9.17s/it]
Layer [9], Head: 0

Layer [9], Head: 1

Layer [9], Head: 2

Layer [9], Head: 3

Layer [9], Head: 4

Layer [9], Head: 5

Layer [9], Head: 6

Layer [9], Head: 7

Layer [9], Head: 8

Layer [9], Head: 9

Layer [9], Head: 10

Layer [9], Head: 11
 50%|██████████████████████▌                      | 2/4 [00:11<00:10,  5.08s/it]
Layer [10], Head: 0

Layer [10], Head: 1

Layer [10], Head: 2

Layer [10], Head: 3

Layer [10], Head: 4

Layer [10], Head: 5

Layer [10], Head: 6

Layer [10], Head: 7

Layer [10], Head: 8

Layer [10], Head: 9

Layer [10], Head: 10

Layer [10], Head: 11
 

In [ ]:
# Print the components with have generated
import json

# Read JSON lines
with open("output_dir/CIFAR10_completeness_top_1500_nouns_5_sentences_imagenet_clean_ViT-L-14-336_algo_svd_data_approx_seed_0_max_text80.jsonl", "r") as json_file:
    for line in json_file:
        entry = json.loads(line)  # Parse each line as a JSON object
        layer = entry["layer"]
        head = entry["head"]
        texts = entry["embeddings_sort"]

        if entry["head"] == -1:
            print(entry.keys())
            print(entry["accuracy"])
        
        print(len(entry["s"]))
        print(f"Layer: {layer}, Head: {head}")
        print("Texts:")
        for text in texts:
            print(text)

In [ ]:
# Go on playground to analyze the results
models = ["ViT-B-32", "ViT-B-16", "ViT-L-14", "ViT-H-14", "ViT-L-14-336"]
algorithms = ["svd_data_approx", "text_span"]
seed = 0
for model in models:
    for algorithm in algorithms:
        with open(f"output_dir/imagenet_completeness_top_1500_nouns_5_sentences_imagenet_clean_ViT-B-32_algo_text_span_seed_0.jsonl", "r") as json_file:

            for line in json_file:
                entry = json.loads(line)  # Parse each line as a JSON object
                layer = entry["layer"]
                head = entry["head"]
                texts = entry["embeddings_sort"]

                if entry["head"] == -1:
                    print(entry.keys())
                    print(entry["accuracy"])